# Import geo-data of "ÖV" in Geneva using swisstopo API

## Libraries and settings

In [1]:
# Libraries
import os
import requests
import json
import folium
from pandas import json_normalize

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/project_arm_airbnb/data_oev


## API request

### Train stations

In [35]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area(id:3601685488);
        node ["railway"="station"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('station.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)

# Number of rows and columns
print(df.shape)

# First rows
df.head()

(3, 32)


,type,id,lat,lon,tags.addr:city,tags.alt_name,tags.ele,tags.internet_access,tags.internet_access:fee,tags.internet_access:operator,...,tags.operator,tags.public_transport,tags.railway,tags.railway:ref,tags.train,tags.uic_name,tags.uic_ref,tags.wheelchair,tags.wikidata,tags.wikipedia
0,node,4890696636,46.210229,6.142622,Genève,Genève-Cornavin,392,wlan,no,SBB,...,CFF,station,station,GE,yes,Genève,8501008,yes,Q800819,fr:Gare de Genève-Cornavin
1,node,6044426066,46.200523,6.164821,NaN,NaN,NaN,NaN,NaN,NaN,...,SBB,station,station,GEV,yes,Genève-Eaux-Vives,8516273,NaN,Q3098155,fr:Gare de Genève-Eaux-Vives
2,node,6886690043,46.192484,6.153701,NaN,NaN,NaN,NaN,NaN,NaN,...,SBB,station,station,CHHO,yes,Genève-Champel,8516272,NaN,Q48610587,fr:Gare de Genève-Champel


### Tram stops

In [ ]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area(id:3601685488);;
        node ["railway"="tram_stop"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('tram.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)

# Number of rows and columns
print(df.shape)

# First rows
df.head()


(0, 0)


""
